In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Oct 24 17:43:59 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
%cd /content/drive/MyDrive/WorkSpace/AI_Research/EmotionTorch/code/

/content/drive/MyDrive/WorkSpace/AI_Research/EmotionTorch/code


In [2]:
!pip install wandb -qqq
!pip install pytorchcv -q

     |████████████████████████████████| 1.9 MB 5.1 MB/s 
     |████████████████████████████████| 166 kB 61.5 MB/s 
     |████████████████████████████████| 182 kB 68.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 71.9 MB/s 
     |████████████████████████████████| 162 kB 73.7 MB/s 
     |████████████████████████████████| 162 kB 59.7 MB/s 
     |████████████████████████████████| 158 kB 71.7 MB/s 
     |████████████████████████████████| 157 kB 70.9 MB/s 
     |████████████████████████████████| 157 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 74.8 MB/s 
     |████████████████████████████████| 157 kB 70.3 MB/s 
     |████████████████████████████████| 157 kB 68.8 MB/s 
     |████████████████████████████████| 157 kB 72.0 MB/s 
     |████████████████████████████████| 157 kB 74.0 MB/s 
     |████████████████████████████████| 156 kB 72.2 MB/s 
     |████████████████████████████████| 532 kB 108 kB/s 


In [3]:
import os
import json
import random
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import imgaug
import torch
import torch.multiprocessing as mp
import numpy as np


seed = 1234
random.seed(seed)
imgaug.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

from utils.datasets.fer2013_ds import FERDataset
# from models.vgg16_cbam import  VGG19_CBAM
from models.resnet_cbam import ResidualNet , cbam_resnet50
from models.vggnet import vgg16_bn, vgg19, vgg19_bn, vgg16
from models.resnet import resnet50
from models.vggnet_cbam import vgg16_cbam, vgg19_cbam
from models.vggnet_cbam_pre import vgg16_cbam_pre, vgg16_bn_cbam_pre, vgg19_bn_cbam_pre, MultiFCVGGnetCBam
from models.test_cbam import TestModel
from models.resmasking import *
from models.BamNetwork import *
from models.New_model import *
from models.ResNetVDSR import resnetvdsr_dropout1

from utils.visualize.show_img import show_image_dataset
from trainer.fer2013_trainer import FER2013_Trainer


In [4]:
print(torch.__version__)

1.12.1+cu113


In [5]:
config_path = "/content/drive/MyDrive/WorkSpace/AI_Research/EmotionTorch/code/configs/config_fer2013.json"

In [6]:
configs = json.load(open(config_path))

In [7]:
train_loader = FERDataset( "train", configs)
val_loader = FERDataset("val", configs)
test_loader_ttau = FERDataset("test", configs, ttau = True, len_tta = 10) 
test_loader = FERDataset("test", configs, ttau = False, len_tta = 10) 

28709
3589
3589
3589


In [8]:
# show_image_dataset(train_ds)

In [9]:
# model = resnet50_cbam()
# if torch.cuda.is_available():
#     model.cuda()

In [10]:
# n_inputs = model.classifier[6].in_features  
# model.classifier[6] = nn.Sequential(
#             nn.Linear(n_inputs, 256), nn.ReLU(), nn.Dropout(0.2),
#             nn.Linear(256, 7))

In [11]:
# import torchvision
# model1 = vgg16_bn()
# model = resnet50(True, True, num_classes = 7)
# model1 = ResidualNet("ImageNet", 50, 7, "CBAM")
# model = cbam_resnet50(in_channels=3, num_classes= 7 )
# model = vgg19()
# model = vgg19_bn()
# model1 = vgg16_bn(pretrained = True, batch_norm = True)
# model = vgg19_cbam(num_classes = 7)
# model = vgg16_bn_cbam_pre(num_classes=7)
# pretrained_model = torchvision.models.vgg16_bn(pretrained=True)
# model = TestModel(pretrained_model)
# model = MultiFCVGGnetCBam(model1)
# model = resmasking_dropout1()
# model = resnetvdsr_dropout1()
model = MyModel_dropout1()
state = torch.load("/content/drive/MyDrive/WorkSpace/AI_Research/EmotionTorch/checkpoints/Fer2013_trainers_V5_test_model22_2022Oct24_02.19")
      
model.load_state_dict(state["net"])

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

<All keys matched successfully>

In [12]:
trainer = FER2013_Trainer(model, train_loader, val_loader, test_loader, test_loader_ttau, configs , wb = True)

In [13]:
trainer.acc_on_test()
trainer.acc_on_test_ttau()

         100%|██████████████████████████████| 3589/3589 [12:58<00:00]


Accuracy on Test_ds: 72.165


AttributeError: ignored

In [ ]:
if configs["distributed"] == 1:
    ngpus = torch.cuda.device_count()
    print(ngpus)
    mp.spawn(trainer.Train_model, nprocs=ngpus, args=())
else:
    trainer.Train_model()

------------SETTING UP WANDB--------------


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


...

 Loss: 0.2236 , Accuracy: 92.22%



         100%|██████████████████████████████| 75/75 [00:58<00:00]

 Val_Loss: 1.1484 , Val_Accuracy: 69.73%



Epoch 3:  46%|██████████████████████▉                           | 275/599 [09:59<11:45]
Traceback (most recent call last):
  File "/content/drive/MyDrive/WorkSpace/AI_Research/EmotionTorch/code/trainer/fer2013_trainer.py", line 349, in Train_model
    self.step_per_train()
  File "/content/drive/MyDrive/WorkSpace/AI_Research/EmotionTorch/code/trainer/fer2013_trainer.py", line 208, in step_per_train
    loss.backward()
  File "/usr/local/lib/python3.7/dist-packages/torch/_tensor.py", line 396, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/__init__.py", line 175, in backward
    allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt


AttributeError: ignored